# Module

##For browser automation

In [ ]:
!pip install google-colab-selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [ ]:
from selenium.webdriver.chrome.options import Options
import google_colab_selenium as gs

# setup chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--lang=en-GB")

# Add your custom options here
driver = gs.Chrome(options=chrome_options)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from bs4 import BeautifulSoup
import urllib.request

In [ ]:
import time
import random

In [ ]:
def get_msWait():
  return random.randint(3000, 5000)

##For handle & store data

In [ ]:
import pandas as pd
import numpy as np

##For DateTime

In [ ]:
from datetime import datetime
from dateutil.parser import parse

In [ ]:
def str_to_date(s):
  return parse(s).date()

##Connect to Driver

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1. Crawl Article

*   The New York Times: https://www.nytimes.com/topic/subject/bitcoin
*   Bitcoin News: https://news.bitcoin.com/



## The NYTimes

In [ ]:
df_NYT = pd.DataFrame(columns=['Date', 'Title'])

In [ ]:
driver.get('https://www.nytimes.com/topic/subject/bitcoin')

In [ ]:
# ------------------------------- Crawling
running = True

while running:
  for element in reversed(driver.find_elements(By.CLASS_NAME, 'css-18yolpw')):
    title = element.find_element(By.CLASS_NAME, 'css-8hzhxf').text
    date = element.find_element(By.CLASS_NAME, 'e15t083i3').text

    if '2013' in date:
      running = False

    if title in df_NYT['Title'].to_list():
      break

    if 'bitcoin' in title.lower() or 'crypto' in title.lower():
      print(len(df_NYT))
      print('\t', str(date), str(title))
      df_NYT.loc[len(df_NYT)] = [date, title]

  driver.find_element(By.TAG_NAME, "html").send_keys(Keys.END)

0
	 Jan. 10, 2024 The Big Bitcoin Tease
1
	 Jan. 10, 2024 S.E.C. Social Media Hack That Sent Bitcoin Soaring Prompts Investigation
2
	 Jan. 10, 2024 Regulators Approve New Type of Bitcoin Fund, in Boon for Crypto Industry
3
	 Jan. 11, 2024 Bitcoin Heads to Wall Street. Now What?
4
	 Jan. 11, 2024 $4 Billion of New Bitcoin Funds Change Hands in First Trading Day
5
	 Jan. 19, 2024 The A.I. Election, Bitcoin’s Wall Street Debut and TikTok’s Doodad Era
6
	 Jan. 19, 2024 Crypto Funds Have Arrived. But Who Needs Them?
7
	 Jan. 19, 2024 Bitcoin E.T.F.s Come With Risks. Here’s What You Should Know.
8
	 Jan. 30, 2024 German Authorities Seize $2 Billion Worth of Bitcoin
9
	 Feb. 3, 2024 Anxiety, Mood Swings and Sleepless Nights: Life Near a Bitcoin Mine
10
	 Feb. 13, 2024 Crypto PAC Jumps Into Senate Race, Opposing Katie Porter in California
11
	 March 5, 2024 Bitcoin Hits Record High, Completing a Stunning Comeback
12
	 March 5, 2024 Is Crypto Back? What to Know About Bitcoin’s Surge.
13
	 Marc

In [ ]:
# ------------------------------- Sort by date
df_NYT['Date'] = df_NYT['Date'].apply(str_to_date)
df_NYT = df_NYT.sort_values('Date')

In [ ]:
# ------------------------------- Remove news date < 2014
df_NYT = df_NYT.drop(df_NYT[df_NYT['Date'] < datetime(2014, 1, 1).date()].index)

In [ ]:
# ------------------------------- Merge title in one day
new_dfNYT = df_NYT.groupby(df_NYT['Date'], as_index=True).agg({'Title': lambda x: '\n'.join(x)}).reset_index()

In [ ]:
# ------------------------------- Info of df
print(new_dfNYT.iloc[0]['Date'])
print(new_dfNYT.iloc[0]['Title'])
new_dfNYT.head(5)

2014-01-08
Five Lessons From Bitcoin
Chinese E-Commerce Giant Alibaba to Ban Bitcoins on Its Sites


,Date,Title
0,2014-01-08,Five Lessons From Bitcoin\nChinese E-Commerce ...
1,2014-01-21,Why Bitcoin Matters
2,2014-01-22,Bitcoin a Fool’s Gold Standard
3,2014-01-24,Bitcoin Is Not Yet Ready for the Real World
4,2014-01-27,Bitcoin Figure Is Accused of Conspiring to Lau...


In [ ]:
# ------------------------------- Saving
df_NYT.to_csv("NYTimes_Bitcoin.csv", index=False)

## Bitcoin News

In [ ]:
df_BNews = pd.DataFrame(columns=['Date', 'Title'])

In [ ]:
i = 0
running = True
to_date = datetime(2014, 1, 1).date()

while running:
  url = 'https://news.bitcoin.com/page/' + str(i) + '/?s=bitcoin'
  driver.get(url)
  time.sleep(get_msWait()/1000)

  news_element = driver.find_elements(By.CLASS_NAME, 'iDoTcb')
  if len(news_element) > 0:
    for element in news_element:
      title = element.find_element(By.CLASS_NAME, 'kDksDk').text
      date = str_to_date(element.find_element(By.CLASS_NAME, 'fMvYEQ').text)

      if date < to_date:
        running = False
        break

      print(len(df_BNews))
      print('\t', str(date), str(title))
      if title not in df_BNews['Title'].to_list():
        df_BNews.loc[len(df_BNews)] = [date, title]
  else:
    running = False

  i = i + 1
  df_BNews.to_csv('/content/drive/MyDrive/BNews.csv')

Streaming output truncated to the last 5000 lines.
19572
	 2021-10-12 Total Value Locked in Defi Surpasses $200 Billion — Fantom, Avalanche, Harmony, Arbitrum TVLs Swell
19573
	 2021-10-12 Major Turkish Crypto Exchange Coinzo Shuts Down
19574
	 2021-10-12 Eightcap Launches Impressive Award-Winning Crypto Derivatives Offering with Ultra-Low Spreads
19575
	 2021-10-12 iExec Launches Oracle Factory Allowing Anyone To Create Custom Oracles in Minutes
19576
	 2021-10-12 Blockchain Intelligence Company Elliptic Raises $60 Million — Evolution Equity Partners, Softbank Bolster Series C
19577
	 2021-10-12 Wormhole Adds Binance Smart Chain Support to Network Bridge
19578
	 2021-10-11 Bank of England: Crypto Assets Pose 'Limited' Risks to Stability of UK Financial System
19579
	 2021-10-11 FTX US Launches NFT Market — Users Can Trade and Mint Solana NFTs, ETH Support on the Way
19580
	 2021-10-11 rhoToken Successfully Launched on Flurry Finance DApp
19581
	 2021-10-11 Mining Hardware Manufacturer

In [ ]:
# ------------------------------- Sort by date
df_BNews = df_BNews.sort_values('Date')

In [ ]:
# ------------------------------- Remove news date < 2014
df_BNews = df_BNews.drop(df_BNews[df_BNews['Date'] < datetime(2014, 1, 1).date()].index)

In [ ]:
# ------------------------------- Remove duplicates
df_BNews = df_BNews.drop_duplicates()

In [ ]:
# ------------------------------- Merge title in one day
new_dfBNews = df_BNews.groupby(df_BNews['Date'], as_index=True).agg({'Title': lambda x: '\n'.join(x)}).reset_index()

In [ ]:
# ------------------------------- Info of df
print(new_dfBNews.iloc[0]['Date'])
print(new_dfBNews.iloc[0]['Title'])
new_dfBNews.tail(5)

2015-01-18
Bitcoin News Roundup - January 18th, 2015


,Date,Title
3055,2024-04-15,Coinbase Challenges SEC’s Definition of 'Inves...
3056,2024-04-16,Norway to Target Cryptocurrency Mining Through...
3057,2024-04-17,Australian Regulator Says Crypto Is a Debentur...
3058,2024-04-18,South Korean Inmate on Funeral Leave Robs Cryp...
3059,2024-04-19,Merlin Chain Launches MERL: A Major Leap Forwa...


In [ ]:
# ------------------------------- Save
df_BNews.to_csv('/content/drive/MyDrive/BNews.csv')

#2. Crawl WhaleWallet


*   https://bitinfocharts.com/top-100-richest-bitcoin-addresses.html